In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None
%config Completer.use_jedi = False # Autocomplete works faster

In [2]:
# Importing data
df = pd.read_csv('./data/occurrence.txt', sep='\t')

/home/davidgp/anaconda3/envs/malifo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (198,237) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df.dtypes

gbifID                  int64
abstract              float64
accessRights          float64
accrualMethod         float64
accrualPeriodicity    float64
                       ...   
level1Name             object
level2Gid              object
level2Name             object
level3Gid              object
level3Name             object
Length: 249, dtype: object

In [11]:
df.columns

Index(['gbifID', 'abstract', 'accessRights', 'accrualMethod',
       'accrualPeriodicity', 'accrualPolicy', 'alternative', 'audience',
       'available', 'bibliographicCitation',
       ...
       'recordedByID', 'identifiedByID', 'level0Gid', 'level0Name',
       'level1Gid', 'level1Name', 'level2Gid', 'level2Name', 'level3Gid',
       'level3Name'],
      dtype='object', length=249)

In [21]:
# Cheking if all values are NaN
df.columns[df.isna().all()]

Index(['abstract', 'accessRights', 'accrualMethod', 'accrualPeriodicity',
       'accrualPolicy', 'alternative', 'audience', 'available',
       'bibliographicCitation', 'conformsTo',
       ...
       'nomenclaturalStatus', 'taxonRemarks', 'distanceAboveSurface',
       'distanceAboveSurfaceAccuracy', 'mediaType', 'subgenusKey',
       'typifiedName', 'relativeOrganismQuantity', 'recordedByID',
       'identifiedByID'],
      dtype='object', length=167)

In [22]:
# Deleting columns where all values are NaN. There some columns with no data because the file was made
# acording to the Darwin Core standard: https://dwc.tdwg.org/terms/
df.dropna(how='all', axis=1, inplace=True)

In [23]:
df.columns

Index(['gbifID', 'identifier', 'license', 'modified', 'references', 'rights',
       'rightsHolder', 'type', 'datasetID', 'institutionCode',
       'collectionCode', 'datasetName', 'ownerInstitutionCode',
       'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy',
       'occurrenceStatus', 'occurrenceRemarks', 'eventDate', 'year', 'month',
       'day', 'samplingProtocol', 'waterBody', 'countryCode', 'locality',
       'verbatimLocality', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'georeferenceSources', 'identifiedBy',
       'dateIdentified', 'taxonID', 'scientificName', 'nameAccordingTo',
       'kingdom', 'phylum', 'class', 'order', 'family', 'genus',
       'specificEpithet', 'infraspecificEpithet', 'taxonRank',
       'taxonomicStatus', 'datasetKey', 'publishingCountry', 'lastInterpreted',
       'elevation', 'elevationAccuracy', 'depth', 'depthAccuracy', 'issue',
       'hasCoordinate', 'hasGeospatialIssues', 'taxonKey', 'acceptedTa

In [49]:
# Checking if there are duplicates
df.drop_duplicates().shape, df.shape

((38324, 82), (38324, 82))

In [56]:
# Let's check issues in the data. Issue column contains general information and hasGeospatialIssues column
# has geospatial problems
df['issue'].value_counts()

COORDINATE_REPROJECTION_SUSPICIOUS;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                     36277
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;COUNTRY_INVALID;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE          1609
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                           120
ZERO_COORDINATE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                                          115
COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH_FUZZY;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                      63
COORDINATE_REPROJECTION_SUSPICIOUS;TAXON_MATCH_NONE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                       44
ZERO_COORDINATE;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                               42
COORDINATE_REPROJECTION_SUSPICIOUS

In [61]:
# It seems that all rows have the issues
# COORDINATE_REPROJECTION_SUSPICIOUS;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE  
# Let's check some issues, like COUNTRY_INVALID, COUNTRY_COORDINATE_MISMATCH, ZERO_COORDINATE,
# TAXON_MATCH_FUZZY, TAXON_MATCH_NONE, TAXON_MATCH_HIGHERRANK, COUNTRY_DERIVED_FROM_COORDINATES

df[df['issue'].str.contains('ZERO_COORDINATE')][['decimalLatitude','decimalLongitude']].value_counts()

decimalLatitude  decimalLongitude
0.0              0.0                 158
dtype: int64

In [58]:
df[df['issue'].str.contains('COUNTRY_INVALID')]['countryCode'].value_counts()

ZZ    1609
BQ       2
SA       1
Name: countryCode, dtype: int64

In [57]:
df[df['issue'].str.contains('COUNTRY_COORDINATE_MISMATCH')]['countryCode'].value_counts()

ZZ    1609
NL      59
SX      22
ZA      18
EG      15
MV      11
AI       5
CU       5
BS       4
AU       4
VI       3
NO       3
VG       2
TR       2
FJ       2
TH       2
EC       1
US       1
DK       1
UM       1
HN       1
Name: countryCode, dtype: int64

## Continuar analizando los valores de la columna issue

In [25]:
df['hasGeospatialIssues'].value_counts()

False    36437
True      1887
Name: hasGeospatialIssues, dtype: int64

In [26]:
df[df['hasGeospatialIssues'] == True]

,gbifID,identifier,license,modified,references,rights,rightsHolder,type,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,recordedBy,occurrenceStatus,occurrenceRemarks,eventDate,year,month,day,samplingProtocol,waterBody,countryCode,locality,verbatimLocality,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferenceSources,identifiedBy,dateIdentified,taxonID,scientificName,nameAccordingTo,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,issue,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,speciesKey,species,genericName,acceptedScientificName,verbatimScientificName,protocol,lastParsed,lastCrawled,repatriated,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
12,2993408634,diveboard:461013_356275_0,CC_BY_NC_4_0,2020-11-28T01:25:24Z,https://www.diveboard.com/epnfrnrush/D2KIvUP,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461013_356275_0,diveboard:461013_356275_0,Epnfrnrush,PRESENT,Non validated citizen science record,2019-12-06T15:14:00,2019,12,6,citizen science scuba dive observation,Carribean Sea,ZZ,Hilma Hooker,"Hilma Hooker, Bonaire",12.1049,-68.2884,100.0,Google Maps,Epnfrnrush,2019-12-06T00:00:00,http://eol.org/pages/356275,"Myleus torquatus (Kner, 1858)",EOL,Animalia,Chordata,Actinopterygii,Characiformes,Serrasalmidae,Myloplus,torquatus,NaN,SPECIES,SYNONYM,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:19.334Z,0.0,0.0,12.190,12.190,COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COO...,True,True,2354468,7193779.0,1,44.0,204.0,537.0,4287106.0,2353403.0,7193779.0,Myloplus torquatus,Myleus,"Myloplus torquatus (Kner, 1858)","Myleus torquatus (Kner, 1858)",DWC_ARCHIVE,2020-12-15T21:04:19.334Z,2020-12-11T15:02:29.629Z,True,BES,"Bonaire, Sint Eustatius and Saba",BES.1_1,Bonaire,NaN,NaN,NaN,NaN
143,2993408503,diveboard:461376_585434_0,CC_BY_NC_4_0,2020-11-28T02:32:39Z,https://www.diveboard.com/epnfrnrush/D1ySW9P,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461376_585434_0,diveboard:461376_585434_0,Epnfrnrush,PRESENT,Non validated citizen science record,2009-12-07T14:31:00,2009,12,7,citizen science scuba dive observation,Caribbean Sea,ZZ,Breezes House Reef,"Breezes House Reef, Curacao",12.0877,-68.9023,100.0,Google Maps,Epnfrnrush,2009-12-07T00:00:00,http://eol.org/pages/585434,"Aulostomus maculatus Valenciennes, 1841",EOL,Animalia,Chordata,Actinopterygii,Syngnathiformes,Aulostomidae,Aulostomus,maculatus,NaN,SPECIES,ACCEPTED,66f6192f-6cc0-45fd-a2d1-e76f5ae3eab2,FR,2020-12-15T21:04:15.269Z,0.0,0.0,9.145,9.145,COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COO...,True,True,2332595,2332595.0,1,44.0,204.0,773.0,5903.0,2332594.0,2332595.0,Aulostomus maculatus,Aulostomus,"Aulostomus maculatus Valenciennes, 1841","Aulostomus maculatus Valenciennes, 1841",DWC_ARCHIVE,2020-12-15T21:04:15.269Z,2020-12-11T15:02:29.629Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,2993408354,diveboard:461377_455703_0,CC_BY_NC_4_0,2020-11-28T02:32:39Z,https://www.diveboard.com/epnfrnrush/D227Aps,http://creativecommons.org/publicdomain/zero/1.0/,Diveboard,Event,https://ipt.diveboard.com/resource.do?r=divebo...,Diveboard,Diveboard,Diveboard - Scuba diving citizen science,Diveboard,HUMAN_OBSERVATION,diveboard:461377_455703_0,diveboard:461377_455703_0,Epnfrnrush,PRESENT,Non validated citizen science record,2009-12-08T15:47:00,2009,12,8,citizen science scuba dive observation,Caribbean Sea,ZZ,Breezes House Reef

In [27]:
df[df['hasGeospatialIssues'] == True]['issue'].value_counts()

COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;COUNTRY_INVALID;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE    1609
COORDINATE_REPROJECTION_SUSPICIOUS;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                     120
ZERO_COORDINATE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                                    115
ZERO_COORDINATE;COUNTRY_COORDINATE_MISMATCH;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                         42
ZERO_COORDINATE;TAXON_MATCH_NONE;INSTITUTION_MATCH_NONE;COLLECTION_MATCH_NONE                                                     1
Name: issue, dtype: int64